## Score en Kaggle: 0.85290

In [1]:
import catboost as cb
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
labels = pd.read_csv('labels_training_set.csv')

In [3]:
processed = pd.read_csv('features_04.csv')

In [4]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [5]:
processed.head()

,person,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
0,0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,...,1362,False,False,True,False,False,True,False,True,False
1,00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,...,1155,False,True,True,False,False,False,True,True,False
2,00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,...,12924,False,True,False,False,False,False,True,True,False
3,000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,...,1155,False,True,True,True,True,True,True,True,False
4,000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,...,12924,True,True,True,True,False,True,False,True,False


In [6]:
features = processed.set_index(processed.columns[0]).drop(['primer_visita','ultima_visita'],axis=1)

In [8]:
dataset = labels.set_index('person').join(features,how='left')

In [9]:
dataset.head()

,label,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,0,viewed product,iPhone 5s,Smartphone,1,17,23,68,9,0,...,865,False,True,True,True,True,True,True,True,False
6ec7ee77,0,visited site,Ninguno,Smartphone,0,1,0,2,1,3,...,12924,False,True,False,False,False,False,False,True,False
abe7a2fb,0,viewed product,iPhone 6,Smartphone,0,22,31,96,109,2,...,12924,True,True,True,True,False,True,True,True,False
34728364,0,viewed product,Lenovo Vibe K5,Smartphone,0,4,24,37,9,4,...,12924,False,False,False,True,False,True,True,True,False
87ed62de,0,viewed product,Samsung Galaxy J7,Computer,0,1,9,17,1,12,...,59,False,True,True,False,False,False,False,True,False


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, 0566e9c1 to 824fa9e3
Data columns (total 35 columns):
label                          19414 non-null int64
mayor_evento                   19414 non-null object
prod_visto                     19414 non-null object
dispos                         19414 non-null object
compras                        19414 non-null int64
visitas                        19414 non-null int64
cant_prod_vistos               19414 non-null int64
cant_eventos                   19414 non-null int64
permanencia                    19414 non-null int64
hace_cuanto_visito             19414 non-null int64
eventos_por_dia                19414 non-null float64
channel                        19414 non-null object
vio_celulares                  19414 non-null bool
region                         19414 non-null object
is_new                         19414 non-null bool
personas_mismo_prod_visto      19414 non-null int64
opsis                          19414 non-null obj

In [11]:
training_features = dataset.drop(['label'],axis=1)
training_labels = dataset['label']

In [12]:
xtrain,xtest,ytrain,ytest = train_test_split(training_features,training_labels)

In [13]:
catBoost = cb.CatBoostClassifier(iterations=200,
                                 depth=8,
                                 l2_leaf_reg=50,
                                 random_strength=0.1,
                                 bagging_temperature=0,
                                 border_count=128)

In [14]:
catBoost.fit(xtrain,ytrain,verbose=0,cat_features=[0,1,2,10,11,12,13,15,16,21,25,26,27,28,29,30,31,32,33])

In [15]:
roc_auc_score(ytest,catBoost.predict_proba(xtest)[:,1])

0.8472876040119464

In [16]:
catBoost.fit(training_features,training_labels,verbose=0,cat_features=[0,1,2,10,11,12,13,15,16,21,25,26,27,28,29,30,31,32,33])

In [17]:
prediction_features = kaggle.set_index('person').join(features,how='left')

In [18]:
pred = catBoost.predict_proba(prediction_features)[:,1]
submit = kaggle.set_index('person')
submit['label'] = pred

In [19]:
submit.head()

,label
person,
4886f805,0.007877
0297fc1e,0.085934
2d681dd8,0.015457
cccea85e,0.059963
4c8a8b93,0.028219


In [20]:
submit.to_csv('submit.csv')